In [1]:
import pandas as pd
import pyensembl

In [2]:
#data.to_csv("~/projects/BIO/novel_junction.20230512.tsv",sep="\t",index=False)
data=pd.read_csv("~/projects/BIO/proteomics_MU_junction_all.20230529.tsv",sep="\t")

prepare data table- merge leafcutter ds results 

In [62]:
# mutated VS unmutated SF3B1 - group 2 - M+U CLL

# read in the data
cluster = pd.read_csv('~/projects/LEAFCUTTER/DS/DS.five_percent/analysis.20230512/fdr0.05/filtered.a2.20230512_cluster_significance.txt',sep=" ")
effect_size=pd.read_csv('~/projects/LEAFCUTTER/DS/DS.five_percent/analysis.20230512/fdr0.05/filtered.a2.20230512_effect_sizes.txt',sep=" ")

In [63]:
# filtered out junction mut(psi) > unmut(psi) --> delta psi < 0
mutated=effect_size[effect_size["deltapsi"]<0]

In [ ]:
mutated[["chr","start","end","cluster"]]=mutated["intron"].str.split(":",expand=True)
mutated["start"]=mutated["start"].astype(int)
mutated["end"]=mutated["end"].astype(int)
cluster[["chr","cluster"]]=cluster["cluster"].str.split(":",expand=True)
mutated=mutated.merge(cluster,on=["chr","cluster"],how="left")


In [65]:
data=mutated.filter(items=["intron","chr","start","end","genes","cluster","p.adjust"])
data.dropna(axis=0,inplace=True)

In [66]:
# gtf=pd.read_csv("/private/resources/gencode19_noChrPrefix_mitoMT.gtf",sep="\t", comment='#', header=None)
# all_intron=pd.read_csv("/home/ls/rachelcw/projects/BIO/annontation_code.20221225_all_introns.bed.gz",sep="\t",header=None,compression="gzip")

In [5]:
all_intron=pd.read_csv("/home/ls/rachelcw/projects/BIO/annontation_code.20221225_all_introns.bed.gz",sep="\t",header=None,compression="gzip",usecols=[0,1,2,3,4,5,6,8],names=["chr","start","end","gene","gene_id","strand","transcript_id","annotation"])
protein_intron=all_intron[all_intron["annotation"]=="protein_coding"]

/home/ls/rachelcw/miniconda3/envs/bio/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# in order to get the genes by individual.
def get_gene_list(genelist):
    data_gene_list=[]
    for value in genelist:
        value = value.split(',')
        data_gene_list.extend(value)
        
    return list(set(data_gene_list))

In [69]:
# filter out intron that are not in the data
data_gene_list=get_gene_list(data['genes'].unique())
intron_in_data=protein_intron[protein_intron["gene"].isin(data_gene_list)]

In [3]:
genome = pyensembl.Genome(
    reference_name='GRCh37',
    annotation_name='my_genome_lab',
    gtf_path_or_url='/home/ls/rachelcw/projects/protein_coding.gtf')
genome.index()

In [71]:
data["novel"]=pd.NA
data["transcript_id"]=pd.NA
for gene in data_gene_list:
    # data_gene=data[data["genes"].str.contains(gene)]
    # intron_gene=intron_in_data[intron_in_data["gene"].str.contains(gene)]
    for index,row in data.iterrows():
        if ((intron_in_data["start"]==row["start"])&(intron_in_data["end"]==row["end"])).any():
            data.loc[index,"novel"]="known"
            trans_id=intron_in_data.loc[(intron_in_data["start"]==row["start"])&(intron_in_data["end"]==row["end"]),"transcript_id"]
            data.loc[index,"transcript_id"]=','.join(trans_id)
        elif (intron_in_data["start"]==row["start"]).any():
            data.loc[index,"novel"]="start"
            trans_id=intron_in_data.loc[intron_in_data["start"]==row["start"],"transcript_id"]
            data.loc[index,"transcript_id"]=','.join(trans_id)
        elif (intron_in_data["end"]==row["end"]).any():
            data.loc[index,"novel"]="end"
            trans_id=intron_in_data.loc[intron_in_data["end"]==row["end"],"transcript_id"]
            data.loc[index,"transcript_id"]=','.join(trans_id)
        else:
            data.loc[index,"novel"]="novel"
            transcript_of_intron=genome.transcript_ids_at_locus(contig=row.chr.replace("chr",""),position=row.end)
            if len(transcript_of_intron)>0:
                data.loc[index,"transcript_id"]=','.join(transcript_of_intron)

        

In [76]:
data[['cluster','strand']]=data['cluster'].str.rsplit('_', expand=True, n=1)

In [75]:
data.dropna(axis=0,inplace=True)

In [8]:
gene_dict={key: value for key, value in zip(protein_intron["gene"].unique(),protein_intron["gene_id"].unique())}

In [13]:
# Duplicate rows based on comma-separated values
new_rows = []
for index, row in data.iterrows():
    values = row['genes'].split(',')
    if len(values) > 1:
        for i, value in enumerate(values):
            new_row = row.copy()
            new_row['genes'] = value
            new_rows.append(new_row)
    else:
        new_rows.append(row)

# Create updated DataFrame
data = pd.DataFrame(new_rows)


In [14]:
data["gene_id"]=data["genes"].map(gene_dict)

In [15]:
# drop gene that arn't "protein coding" 
data.dropna(axis=0,inplace=True)

In [77]:
data.to_csv("~/projects/BIO/proteomics_MU_junction_all.20230530.tsv",sep="\t",index=False)

* get transcript id
* get exons for each transcript
* find the exons around the "new" intron
* check frameshift for the novel exons:
     - if frameshift- translation for each transcript
     - if no frameshift- translation for the first/ the longest transcript

In [119]:
# filter out intron that are known
novel_intron=data[data["novel"]!="known"]

In [24]:
gene_list=get_gene_list(novel_intron["genes"].unique())

create dictionary: key-transcript id, values- exons location of the transcript

transcript_exon: dictionary of the original location.

transcript_exon_updated: dictionary with the novel exons



In [3]:
transcript_exons=dict()
for index, row in novel_intron.iterrows():
    transcript=row["transcript_id"].split(",")
    for t in transcript:
        if t not in transcript_exons:
            exons=genome.exon_ids_of_transcript_id(t)
            locus=[genome.locus_of_exon_id(exon).to_tuple() for exon in exons]
            transcript_exons[t]=locus            
            

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO:root:
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/home/ls/rachelcw/miniconda3/envs/bio/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3552, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-ce3a4c0bb9fc>", line 2, in <module>
    for index, row in novel_intron.iterrows():
NameError: name 'novel_intron' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ls/rachelcw/miniconda3/envs/bio/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2098, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ls/rachelcw/miniconda3/envs/bio/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, numb

In [122]:
transcript_exons_updated=transcript_exons.copy()

In [123]:
import numpy as np
# "novel"= start - we know the end of the exon 
# so we want to find the next exon that is the closest to the end of the intron(start of the next exon) 
# and replace the exon start with the intron end
start_novel_intron=novel_intron[novel_intron["novel"]=="start"]
start_novel_intron["frameshift"]=0
for index, row in start_novel_intron.iterrows():
    transcript=row["transcript_id"].split(",")
    t_frame=[]
    for t in transcript:
        start_exon=np.array([l[1] for l in transcript_exons[t]])
        print(row["end"])
        print(start_exon)
        index_exon=np.abs(start_exon-row["end"]).argmin()
        print(index_exon)
        print("old",transcript_exons[t][index_exon])
        new_exon_locus=tuple([transcript_exons[t][index_exon][0],row["end"],transcript_exons[t][index_exon][2],transcript_exons[t][index_exon][3]])
        transcript_exons_updated[t][index_exon]=new_exon_locus
        if (transcript_exons[t][index_exon][2]-transcript_exons[t][index_exon][1]-1)%3!=0:
            print("new",new_exon_locus)
            t_frame.append(t)
    start_novel_intron.loc[index,"frameshift"]=','.join(t_frame)
        
        


/home/ls/rachelcw/miniconda3/envs/bio/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ls/rachelcw/miniconda3/envs/bio/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


12222394
[12237769 12228228 12226888 12221082 12219792 12215717 12214764 12212231]
3
old ('10', 12221082, 12221131, '-')
new ('10', 12222394, 12221131, '-')
12222394
[12237769 12228228 12226888 12226493 12221082 12219792 12215717 12214764
 12212899 12212716 12209575]
4
old ('10', 12221082, 12221131, '-')
new ('10', 12222394, 12221131, '-')
12222394
[12237769 12228228 12226888 12221082 12219792 12215717 12214764 12212899
 12212716 12211721]
3
old ('10', 12221082, 12221131, '-')
new ('10', 12222394, 12221131, '-')
12222394
[12237769 12228228 12226888 12221028 12219842]
3
old ('10', 12221028, 12221131, '-')
new ('10', 12222394, 12221131, '-')
12222394
[12237769 12228228 12226888 12221082 12219792 12215717 12214764 12212899
 12212716 12207324]
3
old ('10', 12221082, 12221131, '-')
new ('10', 12222394, 12221131, '-')
12222394
[12237769 12228228 12226888 12221082 12219792 12215717 12214768 12212899
 12212716 12209576]
3
old ('10', 12221082, 12221131, '-')
new ('10', 12222394, 12221131, '-')


In [116]:
# "novel"= end - we know the start of the exon 
# so we want to find the previous exon that is the closest to the start of the intron(end of the previous exon) 
# and replace the exon end with the intron start
end_novel_intron=novel_intron[novel_intron["novel"]=="end"]
# start_novel_intron["frameshift"]=0
for index, row in end_novel_intron.iterrows():
    transcript=row["transcript_id"].split(",")
    # t_frame=[]
    for t in transcript:
        end_exon=np.array([l[2] for l in transcript_exons[t]])
        index_exon=np.abs(end_exon-row["start"]).argmin()
        new_exon_locus=tuple([transcript_exons[t][index_exon][0],transcript_exons[t][index_exon][1],row["start"],transcript_exons[t][index_exon][3]])
        transcript_exons_updated[t][index_exon]=new_exon_locus

In [ ]:
# novel = novel - the intron is novel so we want to find the closest exons that the intron is between.
novel_novel_intron=novel_intron[novel_intron["novel"]=="novel"]
# start_novel_intron["frameshift"]=0
for index, row in novel_novel_intron.iterrows():
    transcript=row["transcript_id"].split(",")
    # t_frame=[]
    for t in transcript:
        start_exon=np.array([l[1] for l in transcript_exons[t]])
        end_exon=np.array([l[2] for l in transcript_exons[t]])
        index_start=np.abs(start_exon-row["end"]).argmin()
        index_end=np.abs(end_exon-row["start"]).argmin()
        new_start=tuple([transcript_exons[t][index_start][0],row["end"],transcript_exons[t][index_start][2],transcript_exons[t][index_start][3]])
        new_end=tuple([transcript_exons[t][index_end][0],transcript_exons[t][index_end][1],row["start"],transcript_exons[t][index_end][3]])
        transcript_exons_updated[t][index_start]=new_start
        transcript_exons_updated[t][index_end]=new_end

In [96]:
ex=genome.exon_ids_of_transcript_id("ENST00000596530.1")
# [ for exon in ex]
genome.locus_of_transcript_id("ENST00000596530.1")

Locus(contig='19', start=20718631, end=20727515, strand='-')

In [ ]:
# pyensembl.search.find_nearest_locus(start, end, loci)
# Finds nearest locus (object with method distance_to_interval) to the interval defined by the given start and end positions. Returns the distance to that locus, along with the locus object itself.

In [49]:
non_protein=get_gene_list(novel_intron.loc[novel_intron["transcript_id"]=='',"genes"].unique())

In [60]:
len(novel_intron.iloc[191]["transcript_id"])

0

In [54]:
protein_intron.loc[protein_intron["gene"].isin(non_protein),"gene"].unique()

array(['PLGLB2'], dtype=object)

In [59]:
genome.transcript_ids_at_locus(contig="2",position=88037624)

[]

In [21]:
genome.transcript_ids_of_exon_id('ENSE00000728678.1')

ValueError: No results found for query:

            SELECT distinct transcript_id
            FROM transcript
            WHERE exon_id = ?
        
with parameters: ['ENSE00000728678.1']

In [62]:
genome.transcript_ids_at_locus(contig="10",position=131960844,end=131964771,strand="+")

['ENST00000331244.5', 'ENST00000368644.1', 'ENST00000481034.1']

In [10]:
genome.exon_ids_of_transcript_id("ENST00000368644.1")

['ENSE00002174390.1',
 'ENSE00000728668.1',
 'ENSE00003477570.1',
 'ENSE00000728676.1',
 'ENSE00000728678.1',
 'ENSE00000728681.1',
 'ENSE00000728683.1',
 'ENSE00000728687.1',
 'ENSE00000728691.1',
 'ENSE00000577957.1',
 'ENSE00001217321.1',
 'ENSE00001217252.2']